In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from functools import partial
#
import re
from RootInteractive.TTreeHnInteractive.TTreeHnBrowser import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDraw import *
output_notebook()
import ROOT
ROOT.gSystem.Load("$ALICE_ROOT/lib/libSTAT.so")
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.Tools.histoNDTools import *
import logging
from RootInteractive.Tools.Alice.BetheBloch import *
from bokeh.palettes import *
logging.getLogger().setLevel(1)


In [ ]:
bokehDrawHistoColz2?

# Setup Toy MC

In [ ]:
nPoints=1000000
pdg=ROOT.TDatabasePDG.Instance()
particleList=["e+","mu+","pi+","K+","proton"]
massList=[ pdg.GetParticle(a).Mass() for a in particleList ]    
def GetMass(iPart):
    return [massList[i] for i in iPart]
detectors=["ITS","TPC0","TPC1","TPC2","TRD"]

# Make Toy MC generatinc array of dEdx measurement

In [ ]:
p=np.random.random(nPoints)
p*=5
p+=0.1
particle=np.random.randint(0,5,size=nPoints)
mass=np.asarray(GetMass(particle))
lbg=np.log(p/mass)
data={'p':p, 'particle':particle, 'lbg':lbg}
df=pd.DataFrame(data)
for det in detectors:
    df[det]=BetheBlochAlephNP(lbg)
    df[det]*=np.random.normal(1,0.1,nPoints)

In [ ]:
df.head(10)

In [ ]:
hisString="TRD:p:particle:#TRD>0>>hisTRDPP(100,0.5,3,100,0.3,5,5,0,5)"
histo=makeHistogram(df,hisString)

In [ ]:
#drawHisto(histo,slice):
%timeit -n 100  histo["H"].sum()

In [ ]:
#plt.plot(histo["axes"][0][1:51],histo["H"][:,10,2],"ro")
#histo["axes"][0][0:50]

In [ ]:
bokehDrawHistoSliceColz?

In [ ]:
def bokehDrawHistoColz2(histo, hSlice, axisDraw, axisColor, figOption, graphOption):
    # slice example  = np.index_exp[:, 1:3,3:5]
    sliceString=str(hSlice).replace("slice","")
    TOOLTIPS = [
        ("index", "$index"),
        ("Slice",sliceString)
    ]
    array=histo["H"][hSlice]
    plots=[]

    axisSum=tuple([idx for idx,a in enumerate(array) if idx not in [axisDraw,axisColor]])
    #= ColumnDataSource(dfQuery)
    #bin0=slice[axisColor][0]
    start=hSlice[axisColor].start
    stop=hSlice[axisColor].stop
    step=1
    hSliceList=list(hSlice)
    color=Category10[stop-start+2]
    colorIndex=0
    data={}
    x=histo["axes"][axisDraw][hSlice[axisDraw]]
    data['varX']=x
    fIndex=0
    for a in range(start,stop,step):
        hSliceList[axisColor]=slice(a,a+1,step)
        hSliceLocal=tuple(hSliceList)
        #print a, histo["axes"][axisColor][a]
        y=np.sum(histo["H"][hSliceLocal],axis=(1,2))
        data["varY"+str(fIndex)]=y
        TOOLTIPS.append(("varY"+str(hSliceList[axisColor]).replace("slice",""),"@varY"+str(fIndex)))
        fIndex+=1
    source =ColumnDataSource(data)
    p2 = figure(title=histo["name"], tooltips=TOOLTIPS, **figOption)
    #print(data)
    fIndex=0
    for a in range(start,stop,step):
        p2.scatter("varX", "varY"+str(fIndex),source=source, color=color[fIndex],marker=bokehMarkers[fIndex%4], legend="Bin"+str(a), **graphOption)
        fIndex+=1
    p2.legend.click_policy="hide"   
    show(p2)    
    

In [ ]:
figOption={'plot_width':600, 'plot_height':400}
plotOption={'size':6}
bokehDrawHistoColz2(histo,np.index_exp[1:50, 10:11,0:5], 0,2, {'plot_width':800, 'plot_height':700},{'size':10})

In [ ]:
plotOption={'plot_width':600, 'plot_height':400}
graphOption={'size':2}

def dumpFun(plotOption, kwargs):
    print (plotOption)
    print (kwargs)
    
dumpFun(plotOption,graphOption)

In [ ]:
hSlice = np.index_exp[:, 1:3,3:5]
print(histo["H"][hSlice].shape)
np.sum(histo["H"][hSlice],axis=(1,2))

In [ ]:
"y"+str(1)